In [5]:
import pandas as pd
import sqlalchemy
import sqlite3
import requests
from bs4 import BeautifulSoup
import re
import json

In [6]:
data = pd.read_csv('../data/processed/glob_data.csv')

In [7]:
def transform_data(data):
    relevant_data = data[['country_code', 'normalized_job_code', 'age']]
#    relevant_data = data[['uuid', 'age', 'country_code', 'normalized_job_code']]
    relevant_data = relevant_data[relevant_data['normalized_job_code'].notna()]
    #relevant_data['Quantity'] = relevant_data.groupby('normalized_job_code')['normalized_job_code'].transform('count')
    relevant_data['Quantity'] = relevant_data.groupby('country_code','age')['normalized_job_code'].transform('value_counts')
    relevant_data.rename(columns={'age': 'Age', 'country_code': 'Country', 'normalized_job_code': 'Job Title'},
                       inplace=True)
    relevant_data.set_index('Country')
    return relevant_data

In [8]:
relevant_data = transform_data(data)
#relevant_data = relevant_data[relevant_data['normalized_job_code'].notna()]
relevant_data.head(1)

ValueError: No axis named age for object type <class 'type'>

In [10]:
def country_name_import(relevant_data):
    scrap_url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
    html = requests.get(scrap_url).content
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all('td')

    # Scrap countries list and cleaning some mess
    codes = [i.text.strip() for i in table]
    pre_codes_clean = [i for i in codes if i != ""]
    codes_clean = [re.sub(r'\b[()]|[()]\b', '', code) for code in pre_codes_clean]
    codes_clean[3] = codes_clean[3].replace('EL', 'GR')
    codes_clean[55] = codes_clean[55].replace('UK', 'GB')

    # List to Dict and replace values into the column
    countries_codes = {codes_clean[i + 1]: codes_clean[i] for i in range(0, len(codes_clean), 2)}
    clean_data = relevant_data
    #clean_data.dropna(inplace=True) 
    clean_data['Country'].replace(countries_codes, inplace=True)
    
    return clean_data, countries_codes

In [11]:
clean_data, countries_codes = country_name_import(relevant_data)

In [9]:
#clean_data['Quantity'] = clean_data.apply(lambda row: row.normalized_job_code., axis = 1) 
#clean_data['Quantity3'] = df['Data3'].groupby(df['Date']).transform('count')
#clean_data['Quantity2'] = 1
#clean_data['Quantity'] = clean_data.groupby('normalized_job_code')['normalized_job_code'].transform('count') 

In [17]:
def finding_job(clean_data, job_title):
    
    for code in clean_data['Job Title']:
           
        api_url = f'http://api.dataatwork.org/v1/jobs/' + code
        response = requests.get(api_url).json()

        if response['title'] == job_title:
            key_uuid = response['uuid']
            break
    
    return {key_uuid: job_title}

In [18]:
#Select_jobs triggers the functions and finds the uuid tied to job selected and returns {uuid: job_title}
def job_data(clean_data):
    select_job = finding_job(clean_data, 'Data Scientist')
    key_uuid = list(select_job.keys())[0]
    
    return select_job, key_uuid

In [19]:
select_job, key_uuid = job_data(clean_data)

In [23]:
def job_filtering(clean_data, normalized_job_code, key_uuid):
    job_filter = clean_data['Job Title'] == key_uuid
    filtered_data = clean_data[job_filter]
    filtered_data['Job Title'].replace(select_job, inplace=True)
    
    return filtered_data

In [24]:
filtered_data = job_filtering(clean_data, 'Job Title', key_uuid)


/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [25]:
print(filtered_data[['Country','Job Title']].head(3))

     Country       Job Title
206  Belgium  Data Scientist
214  Belgium  Data Scientist
249  Belgium  Data Scientist


In [17]:
#filtered_data.to_csv(f'../data/processed/filtered_data.csv', index=False)

In [32]:
def country_filter(dataframe, col, value):
    filter_country = dataframe[col] == value
    filtered_country = dataframe[filter_country]
    return filtered_country

In [33]:
#At this point, you could choose Filtering by country or not. ie Spain. If empty, selects all.
def country_filtering(filtered_data, country, countries_codes):
    print(country)
    if country == 'All_countries':
        return filtered_data
    elif country in countries_codes.values():
        filtered_data = country_filter(filtered_data, 'Country', country)
        return filtered_data
    else:
        print('Error: country name not included in the database or type error')

In [53]:
country = 'All_countries'
filtered_data = country_filtering(filtered_data, country, countries_codes)

All_countries


In [54]:

filtered_data = filtered_data.reset_index(drop=True)

In [55]:
filtered_data.head(10)

,Country,Job Title,Age,Quantity,Percentage
0,Spain,Data Scientist,51,67,1.175026
1,Spain,Data Scientist,47,67,1.175026
2,Spain,Data Scientist,40,67,1.175026
3,Spain,Data Scientist,35,67,1.175026
4,Spain,Data Scientist,27,67,1.175026
5,Spain,Data Scientist,40,67,1.175026
6,Spain,Data Scientist,23,67,1.175026


In [56]:
def percentage_func(filtered_data):
    filtered_data['Percentage'] = filtered_data['Quantity'] / len(clean_data) * 100
    return filtered_data


In [57]:
wa = filtered_data.at[0,'Quantity']
wa

67

In [58]:
filtered_data['Percentage'] = filtered_data.at[0,'Quantity'] / len(clean_data) * 100


In [59]:
#https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

In [60]:
filtered_data.head(5)

,Country,Job Title,Age,Quantity,Percentage
0,Spain,Data Scientist,51,67,1.175026
1,Spain,Data Scientist,47,67,1.175026
2,Spain,Data Scientist,40,67,1.175026
3,Spain,Data Scientist,35,67,1.175026
4,Spain,Data Scientist,27,67,1.175026


In [61]:
country = 'All_countries'
filtered_data_all = country_filtering(filtered_data, country, countries_codes)

All_countries


In [62]:
filtered_data_all = filtered_data_all.reset_index(drop=True)

In [63]:
filtered_data_all.head(10)

,Country,Job Title,Age,Quantity,Percentage
0,Spain,Data Scientist,51,67,1.175026
1,Spain,Data Scientist,47,67,1.175026
2,Spain,Data Scientist,40,67,1.175026
3,Spain,Data Scientist,35,67,1.175026
4,Spain,Data Scientist,27,67,1.175026
5,Spain,Data Scientist,40,67,1.175026
6,Spain,Data Scientist,23,67,1.175026


In [28]:
data_testeo = pd.read_csv('../data/processed/filtered_data.csv')
data_testeo.head()


,uuid,age,country_code,normalized_job_code
0,78156750-d949-0133-1a02-0a81e8b09a82,21,Belgium,Data Scientist
1,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,Belgium,Data Scientist
2,f30223b0-d950-0133-5b67-0a81e8b09a82,50,Belgium,Data Scientist
3,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,Belgium,Data Scientist
4,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,Czechia,Data Scientist


In [29]:
len(data_testeo['country_code']) == 'Spain'

False

In [30]:
def percentage_func(col):
    len(data_testo['country_code']) == Spain
    
    return len(data_testeo['uuid'])


In [31]:
data_testeo.rename(columns={'age': 'Age', 'country_code': 'Country', 'normalized_job_code': 'Job Title'}, inplace=True) 
data_testeo.head()

,uuid,Age,Country,Job Title
0,78156750-d949-0133-1a02-0a81e8b09a82,21,Belgium,Data Scientist
1,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,Belgium,Data Scientist
2,f30223b0-d950-0133-5b67-0a81e8b09a82,50,Belgium,Data Scientist
3,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,Belgium,Data Scientist
4,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,Czechia,Data Scientist


In [25]:
def quantity_func(col):
    return col.value_counts()


In [ ]:
['Quantity'] = clean_data.groupby('Job Title')['Job Title','Country', 'Age'].transform('count')
    clean_data = clean_data['Country', 'Age', 'Job Title', 'Quantity']

In [28]:
def analyze(filtered_data):
    result = filtered_data.groupby(['Country', 'Age'])['Job Title'].agg(quantity_func).reset_index()
    print(result)
    return result

In [29]:
result = analyze(data_testeo) 

           Country  Age  Job Title
0          Belgium   20          1
1          Belgium   21          1
2          Belgium   43          1
3          Belgium   50          1
4          Czechia   38          1
5          Denmark   25          1
6          Finland   44          1
7           France   16          1
8           France   21          1
9           France   39          1
10          France   43          1
11          France   44          2
12          France   48          1
13         Germany   35          1
14         Germany   36          2
15         Germany   44          1
16         Germany   48          1
17         Germany   49          1
18         Germany   50          1
19         Germany   51          1
20          Greece   39          1
21          Greece   48          1
22         Ireland   38          1
23           Italy   18          1
24           Italy   27          1
25           Italy   29          1
26           Italy   30          1
27           Italy  

In [15]:
type(result)

pandas.core.frame.DataFrame